In [1]:
import re
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as ss
import matplotlib.pyplot as plt
from matplotlib import style, colors
from matplotlib import gridspec
import random
import string
import math

# matplotlib.style.use('ggplot')
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12, 6)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#### what does multiple means?
* multi_hit means a gene hit by two or more mutations. 

* use the new strelka results

use all calls from strelka: this is prefered since this takes into consideration of all soamtic calls:snvs and indels, high/moderate/low/modifier impacts.

in most papers, they only report mutation in coding regions including splicing site, excluding non-coding regions, i will get these numbers as well. synonymous/non-synonymous mutation are only in coding regions.

a good post about match

https://stackoverflow.com/questions/24958358/use-regular-expression-in-python-to-find-two-strings-in-line

a table details the equivalence of ontology an classic effect description

http://snpeff.sourceforge.net/SnpEff_manual.html

# 1. parsing strelka vcf files

In [2]:
def parse_strelka_vcf(vcf):
    df = pd.read_csv(vcf, comment='#', sep='\t', header=None, low_memory=False)
    patient = vcf.split('/')[4]
    df = df[[0,1,3,4,7]]
    df.columns = ['chr', 'pos', 'ref', 'alt', 'effect']
    df = df[(df['effect'].str.contains("HIGH"))|(df['effect'].str.contains("MODERATE"))|(df['effect'].str.contains("LOW"))]
    if not df.empty:
        df['impact'], df['impact_type'], df['gene'] = df['effect'].apply(lambda x: parse_effect(x)).str.split('@').str
        df['patient'] = patient
    df = df.drop('effect', axis=1)
    return df

In [3]:
# keep it easy for now pick HIGH, MODERATE and then LOW
def parse_effect(line):
    effs = line.split('EFF=')[1].split(',')
    #     extract impact, impact_type and gene
    effs = ['@'.join(list(np.array(re.split('\(|\|',ef))[[0,1,6]])) for ef in effs if ('HIGH' in  ef) or ('MODERATE' in ef) or ('LOW' in ef)]
    effs = list(set(effs))
    high = [ef for ef in effs if 'HIGH' in ef]
    moderate = [ef for ef in effs if 'MODERATE' in ef]
    low = [ef for ef in effs if 'LOW' in ef]
    if high:
        anno = high[0]
    elif moderate:
        anno = moderate[0]
    elif low:
        anno = low[0]
    else:
        print('ERROR!')
#     make sure the genes have the same name
    genes = [ef.split('@')[2] for ef in effs]
    
    return anno


In [4]:
# # example run
# ft = '/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02020/hg19a/GSH/A37247_A56468/strelka/23223/bwa/results/passed.somatic.snvs.eff.dbSNP_v137.cosmic_v64.annotations.vcf'
# # assert count_coding_mutations(ft) == 266
# df = parse_strelka_vcf(ft)

# 2. Scope: genes and cohort size

In [4]:
# 118 patients
f = '/projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/118_patients.txt'
with open(f) as file:
    patients = [line.strip() for line in file]
assert len(patients) == 118

In [5]:
# 12 SMGS
f2 = '/projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/118_patients/smgs_reviewed.txt'
with open (f2) as file:
    genes = [line.strip() for line in file]
assert len(genes) == 12

# 3. Strelka vcf files

In [5]:
# f1 = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients/124_patients_bam_vcf.txt'
f1 = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients_new/124_patients_bam_vcf_20190115.csv'
df = pd.read_csv(f1, index_col='patient')
df.head(2)

,HIV_status,DNA_lib,DNA_single_vcf,strelka_indel_vcf,strelka_snv_vcf,source,status,RNA_lib,ribodepleted_lib,DNA_bam,...,other_vcf,DNA_tc,RNA_tc,cnv,bbt_transcriptome,bbt_genome,bbt_transcriptome_other_bacterial,bbt_genome_other_bacterial,bbt_transcriptome_other_viral,bbt_genome_other_viral
patient,,,,,,,,,,,,,,,,,,,,,
HTMCP-03-06-02001,Positive,A37234,/projects/analysis31/A37234/vcall21989_merge13...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,HTMCP_124,Malignant,A37700,A56295,/projects/analysis/analysis22/A37234/merge_bwa...,...,NaN,55.0,55.0,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...
HTMCP-03-06-02002,Negative,A37235,/projects/analysis31/A37235/vcall21991_merge13...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,HTMCP_125,Malignant,A37701,A56296,/projects/analysis/analysis22/A37235/merge_bwa...,...,NaN,70.0,70.0,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...


In [6]:
ndf = df[['strelka_snv_vcf', 'strelka_indel_vcf']].dropna()
ndf.head(2)

,strelka_snv_vcf,strelka_indel_vcf
patient,,
HTMCP-03-06-02001,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...
HTMCP-03-06-02002,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...


In [7]:
ndf = ndf.reindex(patients)

In [8]:
ndf.shape

(118, 2)

In [9]:
for i in ndf['strelka_indel_vcf'].head(2):
    print(i)

/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02001/hg19a/GSH/A37234_A37159/strelka/59844/bwa/results/passed.somatic.indels.eff.dbSNP_v149.cosmic_v82.clinvar_20170801.annotations.classic.vcf
/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02002/hg19a/GSH/A37235_A37160/strelka/59848/bwa/results/passed.somatic.indels.eff.dbSNP_v149.cosmic_v82.clinvar_20170801.annotations.classic.vcf


# 4. concatenate snvs and indels 

In [14]:
dfmg = pd.DataFrame()
for ix, row in ndf.iterrows():
    patient = ix
    snv_vcf = row['strelka_snv_vcf']
    indel_vcf = row['strelka_indel_vcf']
    snv_df = parse_strelka_vcf(snv_vcf)
    indel_df = parse_strelka_vcf(indel_vcf)
    mdf = pd.concat([snv_df, indel_df], sort=False)
    mdf = mdf.drop_duplicates(keep='first')

    if dfmg.empty:
        dfmg = mdf
    else:
        dfmg = pd.concat([dfmg, mdf], sort=False)


In [15]:
dfmg.head()
dfmg.shape

,chr,pos,ref,alt,impact,impact_type,gene,patient
39,1,2706109,C,G,SYNONYMOUS_CODING,LOW,TTC34,HTMCP-03-06-02001
58,1,4771973,C,T,NON_SYNONYMOUS_CODING,MODERATE,AJAP1,HTMCP-03-06-02001
78,1,6146051,G,C,NON_SYNONYMOUS_CODING,MODERATE,KCNAB2,HTMCP-03-06-02001
83,1,6579534,C,T,NON_SYNONYMOUS_CODING,MODERATE,PLEKHG5,HTMCP-03-06-02001
145,1,12170207,C,G,NON_SYNONYMOUS_CODING,MODERATE,TNFRSF8,HTMCP-03-06-02001


(36739, 8)

In [16]:
# subset 12 genes of interest
# dfs = dfmg[dfmg.gene.isin(genes)&dfmg.patient.isin(patients)]
dfs = dfmg[dfmg.patient.isin(patients)]
dfs.head(2)
dfs.shape

,chr,pos,ref,alt,impact,impact_type,gene,patient
39,1,2706109,C,G,SYNONYMOUS_CODING,LOW,TTC34,HTMCP-03-06-02001
58,1,4771973,C,T,NON_SYNONYMOUS_CODING,MODERATE,AJAP1,HTMCP-03-06-02001


(36739, 8)

In [17]:
of = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients_new/strelka_variants_HML_20190521.txt'
dfs.to_csv(of, sep='\t', index=False)

In [61]:
def format_impact(impact):
    impact = impact.upper()
    if '+' in impact:
        impact = impact.split('+')[0]
    return impact

In [62]:
# These are not reasonable, it will remove multi-hits
dfs['impact_new'] = dfs['impact'].apply(format_impact)
dfs.drop('impact', axis=1, inplace=True)
dfs.shape
dfs[dfs.duplicated()]

# dfs.drop_duplicates(keep='first', inplace=True)
# dfs.shape

/projects/da_workspace/software/miniconda/envs/dlpy3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/projects/da_workspace/software/miniconda/envs/dlpy3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


(171, 8)

,chr,pos,ref,alt,impact_type,gene,patient,impact_new


In [63]:
dfs['impact_new'].unique()

array(['NON_SYNONYMOUS_CODING', 'FRAME_SHIFT', 'STOP_GAINED',
       'SYNONYMOUS_CODING', 'SPLICE_SITE_ACCEPTOR', 'SPLICE_SITE_REGION',
       'START_GAINED', 'CODON_DELETION',
       'CODON_CHANGE_PLUS_CODON_DELETION'], dtype=object)

In [64]:
dfs.drop_duplicates().shape

(171, 8)

In [65]:
dfs = dfs.drop_duplicates().groupby(['gene', 'patient']).agg({'impact_new': ','.join}).reset_index()
dfs.head()

,gene,patient,impact_new
0,CASP8,HTMCP-03-06-02001,FRAME_SHIFT
1,CASP8,HTMCP-03-06-02012,STOP_GAINED
2,CASP8,HTMCP-03-06-02036,NON_SYNONYMOUS_CODING
3,CASP8,HTMCP-03-06-02239,NON_SYNONYMOUS_CODING
4,CASP8,HTMCP-03-06-02260,FRAME_SHIFT


In [66]:
def impact_type(x):
#     print(x)
    xsplit = x.split(',')
#     print(x)
#     type = xsplit[0].upper()
    if len(xsplit) > 1:
        type = 'Multi_hit'
    elif len(xsplit) == 1:
        type = xsplit[0]
        if type == 'MISSENSE_VARIANT' or type == 'NON_SYNONYMOUS_CODING':
           type = 'Non-synonymous'
        elif type == 'SYNONYMOUS_VARIANT' or type == 'SYNONYMOUS_CODING':
           type = 'Synonymous'
        elif  type == 'SPLICE_SITE_ACCEPTOR' or  type == 'SPLICE_SITE_DONOR' or type == 'SPLICE_SITE_REGION':
           type = 'Splice site'
        elif type == 'STOP_LOST':
           type = 'Stop lost'
        elif type == 'STOP_GAINED':
           type = 'Stop gained'
        elif  type == 'START_LOST':
           type = 'Start lost'
        elif type == 'START_GAINED':
           type = 'Start gained'
        elif type == 'FRAME_SHIFT' or type == 'FRAMESHIFT_VARIANT':
           type = 'Frameshift'
        elif type == 'INFRAME_DELETION' or type == 'CODON_INSERTION' or type == 'CODON_DELETION' or type == 'CODON_CHANGE_PLUS_CODON_INSERTION' or type == 'CODON_CHANGE_PLUS_CODON_DELETION' or type.lower() == 'disruptive_inframe_deletion':
           type = 'Codon indel'                
        else: exit(1);print('ERROR');print(x)
    return type

In [67]:
dfs['impact'] = dfs['impact_new'].apply(lambda x: impact_type(x))
dfs.shape
dfs = dfs[['gene', 'patient', 'impact']].drop_duplicates()
dfs.shape
dfs = dfs.set_index(['gene', 'patient'])['impact'].unstack()
dfs.head(2)
dfs.shape

(139, 4)

(139, 3)

patient,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02006,HTMCP-03-06-02012,HTMCP-03-06-02013,HTMCP-03-06-02020,HTMCP-03-06-02036,HTMCP-03-06-02040,HTMCP-03-06-02042,HTMCP-03-06-02047,...,HTMCP-03-06-02344,HTMCP-03-06-02346,HTMCP-03-06-02354,HTMCP-03-06-02392,HTMCP-03-06-02411,HTMCP-03-06-02427,HTMCP-03-06-02428,HTMCP-03-06-02435,HTMCP-03-06-02441,HTMCP-03-06-02447
gene,,,,,,,,,,,,,,,,,,,,,
CASP8,Frameshift,NaN,NaN,Stop gained,NaN,NaN,Non-synonymous,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FAT1,NaN,NaN,NaN,NaN,NaN,NaN,Stop gained,Synonymous,NaN,NaN,...,NaN,NaN,Stop gained,NaN,NaN,Frameshift,Multi_hit,Non-synonymous,NaN,NaN


(12, 80)

# 5. add in covariates

In [99]:
# get meta track for example histology
f3 ='/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients/mutation_load_clinic.txt'
ddf = pd.read_csv(f3, sep='\t')
ddf = ddf[['patient', 'HIV_status', 'Putative_histology']]
edf = ddf.set_index('patient').T
edf.loc['Putative_histology',].unique()
edf = edf[patients]
edf.head(2)
edf.shape

array(['Squamous', 'Adenosquamous', 'Adeno', 'Neuroendocrine'],
      dtype=object)

patient,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02003,HTMCP-03-06-02006,HTMCP-03-06-02008,HTMCP-03-06-02012,HTMCP-03-06-02013,HTMCP-03-06-02020,HTMCP-03-06-02036,HTMCP-03-06-02037,...,HTMCP-03-06-02417,HTMCP-03-06-02424,HTMCP-03-06-02427,HTMCP-03-06-02428,HTMCP-03-06-02434,HTMCP-03-06-02435,HTMCP-03-06-02437,HTMCP-03-06-02441,HTMCP-03-06-02442,HTMCP-03-06-02447
HIV_status,Positive,Negative,Positive,Negative,Negative,Negative,Negative,Positive,Negative,Negative,...,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive
Putative_histology,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,...,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Adeno,Squamous,Squamous


(2, 118)

In [97]:
s = (edf.loc['Putative_histology',:].isin(['Adenosquamous', 'Neuroendocrine']))
s[s]
edf[s[s].index]

patient
HTMCP-03-06-02047    True
HTMCP-03-06-02110    True
HTMCP-03-06-02176    True
HTMCP-03-06-02203    True
HTMCP-03-06-02216    True
HTMCP-03-06-02235    True
Name: Putative_histology, dtype: bool

patient,HTMCP-03-06-02047,HTMCP-03-06-02110,HTMCP-03-06-02176,HTMCP-03-06-02203,HTMCP-03-06-02216,HTMCP-03-06-02235
HIV_status,Negative,Positive,Negative,Negative,Positive,Positive
Putative_histology,Neuroendocrine,Adenosquamous,Adenosquamous,Adenosquamous,Adenosquamous,Neuroendocrine


In [98]:
cases other than squamous or adeno
# patient	HIV_status	Histology	APOBEC_signature	Clinical_stage	Age_at_diagnosis
# HTMCP-03-06-02235	Positive	Neuroendocrine	weak	Stage IV	<45
# HTMCP-03-06-02110	Positive	Adenosquamous	weak	Stage IV	<45
# HTMCP-03-06-02216	Positive	Adenosquamous	weak	Stage III	45-65
# HTMCP-03-06-02203	Negative	Adenosquamous	moderate	Stage II	45-65
# HTMCP-03-06-02047	Negative	Neuroendocrine	weak	Stage IV	45-65
# HTMCP-03-06-02176	Negative	Adenosquamous	moderate	Stage III	45-65

In [100]:
dfs = pd.concat([edf, dfs], sort=False)

In [101]:
dfs.shape

(14, 118)

In [102]:
dfs = dfs[patients]

# 6. add in mutation rate (fraction of patients)

In [104]:
dfs['occurrence'] = (dfs.notnull().sum(axis=1)/(dfs.shape[1] -2))
dfs['percentage'] = ['{0}({1}%)'.format(i[0], int(round(i[1]*100))) for i in zip(dfs.index.tolist(), dfs.occurrence)]
# dfs.loc['HIV_status', 'percentage'] = 'HIV_Status'
# dfs.loc['Putative_histology', 'percentage' ] = 'Puatative_histology'
sdf = dfs.drop('occurrence', axis=1).set_index('percentage', drop=True)

In [105]:
sdf

,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02003,HTMCP-03-06-02006,HTMCP-03-06-02008,HTMCP-03-06-02012,HTMCP-03-06-02013,HTMCP-03-06-02020,HTMCP-03-06-02036,HTMCP-03-06-02037,...,HTMCP-03-06-02417,HTMCP-03-06-02424,HTMCP-03-06-02427,HTMCP-03-06-02428,HTMCP-03-06-02434,HTMCP-03-06-02435,HTMCP-03-06-02437,HTMCP-03-06-02441,HTMCP-03-06-02442,HTMCP-03-06-02447
percentage,,,,,,,,,,,,,,,,,,,,,
HIV_status(102%),Positive,Negative,Positive,Negative,Negative,Negative,Negative,Positive,Negative,Negative,...,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive
Putative_histology(102%),Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,...,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Adeno,Squamous,Squamous
CASP8(7%),Frameshift,NaN,NaN,NaN,NaN,Stop gained,NaN,NaN,Non-synonymous,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FAT1(19%),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stop gained,NaN,...,NaN,NaN,Frameshift,Multi_hit,NaN,Non-synonymous,NaN,NaN,NaN,NaN
FBXW7(10%),NaN,NaN,NaN,Non-synonymous,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Non-synonymous,NaN,Frameshift,NaN,NaN,NaN,NaN
MAPK1(5%),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Non-synonymous,NaN,NaN,NaN,NaN,NaN,NaN
MLL2(15%),NaN,Stop gained,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Frameshift,NaN,NaN,NaN,NaN
PCDHA9(3%),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PCDHGA12(5%),NaN,NaN,NaN,Non-synonymous,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Non-synonymous,NaN,NaN,NaN,NaN


In [106]:
of = '/projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/118_patients/smgs_reviewed_details_20190326.txt'
# sdf.to_csv(of, sep='\t')

In [108]:
sdf[:2]

,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02003,HTMCP-03-06-02006,HTMCP-03-06-02008,HTMCP-03-06-02012,HTMCP-03-06-02013,HTMCP-03-06-02020,HTMCP-03-06-02036,HTMCP-03-06-02037,...,HTMCP-03-06-02417,HTMCP-03-06-02424,HTMCP-03-06-02427,HTMCP-03-06-02428,HTMCP-03-06-02434,HTMCP-03-06-02435,HTMCP-03-06-02437,HTMCP-03-06-02441,HTMCP-03-06-02442,HTMCP-03-06-02447
percentage,,,,,,,,,,,,,,,,,,,,,
HIV_status(102%),Positive,Negative,Positive,Negative,Negative,Negative,Negative,Positive,Negative,Negative,...,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive
Putative_histology(102%),Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,...,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Adeno,Squamous,Squamous


In [127]:
pd.unique(sdf.values.ravel())

array(['Positive', 'Negative', 'Squamous', 'Neuroendocrine', 'Adeno',
       'Adenosquamous', 'Frameshift', nan, 'Stop gained',
       'Non-synonymous', 'Synonymous', 'Multi_hit', 'Start gained'],
      dtype=object)

In [109]:
df = sdf.copy()

In [110]:
dftmp = pd.DataFrame([list(np.array(re.split('\(|\)', i))[[0,1]]) for i in df.index.tolist()], columns=['gene', 'mfreq'])

df['gene'] = dftmp.gene.tolist()
df['mfreq'] = dftmp.mfreq.tolist()

df['mfreq'] = df['mfreq'].apply(lambda x: int(x.replace('%', '')))

In [112]:
df = df.sort_values('mfreq', ascending=False)

df.set_index('gene', drop=True, inplace=True)

df.head()

,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02003,HTMCP-03-06-02006,HTMCP-03-06-02008,HTMCP-03-06-02012,HTMCP-03-06-02013,HTMCP-03-06-02020,HTMCP-03-06-02036,HTMCP-03-06-02037,...,HTMCP-03-06-02424,HTMCP-03-06-02427,HTMCP-03-06-02428,HTMCP-03-06-02434,HTMCP-03-06-02435,HTMCP-03-06-02437,HTMCP-03-06-02441,HTMCP-03-06-02442,HTMCP-03-06-02447,mfreq
gene,,,,,,,,,,,,,,,,,,,,,
HIV_status,Positive,Negative,Positive,Negative,Negative,Negative,Negative,Positive,Negative,Negative,...,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,102
Putative_histology,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,...,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Adeno,Squamous,Squamous,102
PIK3CA,Multi_hit,Non-synonymous,NaN,NaN,NaN,Multi_hit,Non-synonymous,NaN,Non-synonymous,NaN,...,NaN,NaN,Non-synonymous,NaN,NaN,NaN,NaN,NaN,Non-synonymous,35
FAT1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stop gained,NaN,...,NaN,Frameshift,Multi_hit,NaN,Non-synonymous,NaN,NaN,NaN,NaN,19
MLL2,NaN,Stop gained,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Frameshift,NaN,NaN,NaN,NaN,15


In [114]:
dfa = (df.loc['HIV_status',] == 'Negative') 
dfa = df[dfa[dfa].index]
df['hiv_neg_mfreq'] = (dfa.notnull().sum(axis=1)/dfa.shape[1]*100).astype(int)
dfb = (df.loc['HIV_status',] == 'Positive') 
dfb = df[dfb[dfb].index]
df['hiv_pos_mfreq'] = (dfb.notnull().sum(axis=1)/dfb.shape[1]*100).astype(int)

In [ ]:
df = df.sort_values(df.index.tolist(), axis=1)

In [157]:
df['percentage'] = ['{0}({1}%)'.format(i[0], i[1]) for i in [(i[0], i[1]) for i in zip(df.index, df.mfreq)] ]

In [159]:
df.set_index('percentage', drop=True, inplace=True)

In [162]:
f2 = '/projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/118_patients/smgs_reviewed_details_with_frequency_20190326.txt'
df.to_csv(f2)